In [1]:
import pandas as pd

data = {'Color' : ['R', 'G', 'B', 'G', 'B', 'R', 'R', 'R']}

df = pd.DataFrame(data)
df

,Color
0,R
1,G
2,B
3,G
4,B
5,R
6,R
7,R


In [2]:
#라벨 인코딩
from sklearn.preprocessing import LabelEncoder

#원핫 인코딩
from sklearn.preprocessing import OneHotEncoder

In [3]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.8 MB/s eta 0:00:00


In [4]:
#바이너리, 프리퀀시 인코더를 가져다 쓰기 위해 임포트
import category_encoders as ce

In [5]:
#라벨 인코더 정의
le = LabelEncoder()

#트랜스폼
result = le.fit_transform(df)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
result

array([2, 1, 0, 1, 0, 2, 2, 2])

In [7]:
view = pd.concat([df, pd.DataFrame(result)], axis=1)
view

,Color,0
0,R,2
1,G,1
2,B,0
3,G,1
4,B,0
5,R,2
6,R,2
7,R,2


In [8]:
#데이터가 희소하다 : 의미있는 데이터가 매우 적다. 0이 많고, 1이 적은 경우 'sparse'
oh = OneHotEncoder(sparse_output=False)

oh_result = oh.fit_transform(df)

In [9]:
oh_result

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [10]:
one_hot = pd.DataFrame(oh_result, columns=['B', 'G', 'R'])
one_hot

,B,G,R
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0
5,0.0,0.0,1.0
6,0.0,0.0,1.0
7,0.0,0.0,1.0


In [11]:
view2 = pd.concat([df, one_hot], axis=1)
view2

,Color,B,G,R
0,R,0.0,0.0,1.0
1,G,0.0,1.0,0.0
2,B,1.0,0.0,0.0
3,G,0.0,1.0,0.0
4,B,1.0,0.0,0.0
5,R,0.0,0.0,1.0
6,R,0.0,0.0,1.0
7,R,0.0,0.0,1.0


In [12]:
df

,Color
0,R
1,G
2,B
3,G
4,B
5,R
6,R
7,R


In [13]:
data =  {'Color' : ['R', 'G', 'B', 'G', 'B', 'R', 'R', 'R', 'W']}

df = pd.DataFrame(data)

In [14]:
#category_encoder

#R = 1(0 1), G =2(1, 0),  B=3(1, 1) , W=4(1, 0, 0)=>
BE = ce.BinaryEncoder(cols=['Color'])
result_b = BE.fit_transform(df)

print(result_b)

   Color_0  Color_1  Color_2
0        0        0        1
1        0        1        0
2        0        1        1
3        0        1        0
4        0        1        1
5        0        0        1
6        0        0        1
7        0        0        1
8        1        0        0


In [15]:
#라벨, 원핫, 바이너리 -> 문자를 숫자로 바꾸되, 숫자에 어떤 의미가 있으면? "안..된다..."
#프리퀀시 인코딩 -> 문자를 숫자로 바꾸는데, '의미가 있어' -> 얼마나 자주 출현했냐.

#날씨와 교통량 데이터를 분석. 비오는 - 대중교통/택시 => 맑은날 대중교통/택시
FE = ce.CountEncoder(cols=['Color'])
result_f = FE.fit_transform(df)
print(result_f)

   Color
0      4
1      2
2      2
3      2
4      2
5      4
6      4
7      4
8      1


In [16]:
view3 = pd.concat([df, result_f], axis=1)
view3

,Color,Color
0,R,4
1,G,2
2,B,2
3,G,2
4,B,2
5,R,4
6,R,4
7,R,4
8,W,1


In [17]:
#정규화....0~1사이의 수로 바꿔줌(빈도수 자체가 너무 큰 숫자일때..)
#숫자가 너무 크거나, 전체 중에 출현빈도의 %로 보고싶을 때 유용
frequency_encoded = df['Color'].value_counts(normalize=True)

#.map(함수, 메서드) -> 데이터프레임의 각 행에다가 함수를 적용해라.
df['color_fn'] = df['Color'].map(frequency_encoded)
print(df)

  Color  color_fn
0     R  0.444444
1     G  0.222222
2     B  0.222222
3     G  0.222222
4     B  0.222222
5     R  0.444444
6     R  0.444444
7     R  0.444444
8     W  0.111111


In [19]:
#데이터를 특정 범위로 변환함
#데이터의 최소값과 최댓값을 이미 알고 있거나, 데이터를 특정 범위 내로 변환할 때 유용
from sklearn.preprocessing import MinMaxScaler

#데이터의 평균을 제거하고, 분산을 1로 만들어서 표준 정규 분포(평균 0, 표준편자 1)로 변환
#데이터가 정규 분포(가우시안 분포) 모양을 따른다고 가정할 때 유용
from sklearn.preprocessing import StandardScaler

#중앙값과 IQR(사분위 수 범위)을 사용하여 데이터를 변환
#데이터에 이상치가 포함된 경우에 유용(중앙값을 사용하므로 이상치의 영향을 최소화함)
from sklearn.preprocessing import RobustScaler

In [21]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np

raw = load_wine()
X = pd.DataFrame(raw.data, columns = raw.feature_names)
Y = pd.DataFrame(raw.target, columns=['target'])

In [25]:
#스탠다드 스케일러
#정의
s_scaler = StandardScaler()

#핏_트랜스폼
result = s_scaler.fit_transform(X[['alcohol', 'magnesium']])

#새로운 df
print(result)

[[ 1.51861254  1.91390522]
 [ 0.24628963  0.01814502]
 [ 0.19687903  0.08835836]
 [ 1.69154964  0.93091845]
 [ 0.29570023  1.28198515]
 [ 1.48155459  0.86070511]
 [ 1.71625494 -0.26270834]
 [ 1.3086175   1.49262517]
 [ 2.25977152 -0.192495  ]
 [ 1.0615645  -0.12228166]
 [ 1.3580281   0.36921172]
 [ 1.38273339 -0.33292168]
 [ 0.92568536 -0.75420173]
 [ 2.16095032 -0.61377505]
 [ 1.70390229  0.1585717 ]
 [ 0.77745356  0.86070511]
 [ 1.60508109  1.42241183]
 [ 1.02450655  1.07134513]
 [ 1.46920194  0.57985175]
 [ 0.78980621  1.14155847]
 [ 1.3086175   1.84369188]
 [-0.08723191  0.1585717 ]
 [ 0.87627476  0.08835836]
 [-0.18605311 -0.33292168]
 [ 0.61686912 -0.26270834]
 [ 0.06099988  1.7032652 ]
 [ 0.48098997 -0.47334836]
 [ 0.36981612 -0.40313502]
 [ 1.07391715  0.50963841]
 [ 1.2592069  -0.26270834]
 [ 0.90098006  0.08835836]
 [ 0.71569031  0.43942506]
 [ 0.83921681  0.29899838]
 [ 0.93803801  2.26497192]
 [ 0.62922177  0.72027843]
 [ 0.59216382  0.01814502]
 [ 0.34511082  0.72027843]
 

In [30]:
mScaler = MinMaxScaler(feature_range=(0, 10))

result = mScaler.fit_transform(X)
print(result)

[[8.42105263 1.91699605 5.72192513 ... 4.55284553 9.70695971 5.61340942]
 [5.71052632 2.05533597 4.17112299 ... 4.63414634 7.8021978  5.5064194 ]
 [5.60526316 3.20158103 7.00534759 ... 4.47154472 6.95970696 6.46932953]
 ...
 [5.89473684 6.99604743 4.81283422 ... 0.89430894 1.06227106 3.97289586]
 [5.63157895 3.65612648 5.40106952 ... 0.97560976 1.28205128 4.0085592 ]
 [8.15789474 6.64031621 7.37967914 ... 1.05691057 1.20879121 2.01141227]]


In [ ]:
# 1 5 3 1 5 5 9 4 7 6 4
# 다음 자료의 사분위수 범위 구하세요

- 1 [3 (Q1)] 4 [5 (Q2)] 6 [7 (Q3)] 9
- IQR = 7 - 3 = 4
- 낮은 이상치 = 3 * -1.5
- 높은 이상치 = 7 * 1.5

In [31]:
rscaler = RobustScaler()

result = rscaler.fit_transform(X)
print(result)

[[ 0.8973384  -0.10472973  0.20143885 ...  0.22222222  0.92494929
   0.80804954]
 [ 0.11406844 -0.05743243 -0.63309353 ...  0.25185185  0.5030426
   0.77708978]
 [ 0.08365019  0.33445946  0.89208633 ...  0.19259259  0.31643002
   1.05572755]
 ...
 [ 0.16730038  1.63175676 -0.28776978 ... -1.11111111 -0.98985801
   0.33333333]
 [ 0.09125475  0.48986486  0.02877698 ... -1.08148148 -0.94117647
   0.34365325]
 [ 0.82129278  1.51013514  1.09352518 ... -1.05185185 -0.95740365
  -0.23426213]]
